In [1]:
!pip install ta

     |████████████████████████████████| 15.0MB 13.0MB/s eta 0:00:01   |██████                          | 2.9MB 2.6MB/s eta 0:00:05     |███████▌                        | 3.5MB 2.6MB/s eta 0:00:05
  Using cached https://files.pythonhosted.org/packages/39/b7/441375a152f3f9929ff8bc2915218ff1a063a59d7137ae0546db616749f9/pandas-0.25.0-cp37-cp37m-macosx_10_9_x86_64.macosx_10_10_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/aa/7d/6c71c35c201f6d5cec318c7ed7841317adbf291513742865ed8904ae4ea9/scikit_learn-0.21.2-cp37-cp37m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/3d/73/fe30c2daaaa0713420d0382b16fbb761409f532c56bdcc514bf7b6262bb6/pytz-2019.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/04/66/ec5f1283d6a290a9153881a896837487338c44639c1305cc59e1c7b69cc9/scipy-1.3.0-cp37-cp37m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_1

In [154]:
!pip install datetime --upgrade

Requirement already up-to-date: datetime in /Users/dwightchurchill/.local/share/virtualenvs/dwightchurchill-Bl87FlYt/lib/python3.7/site-packages (4.3)


In [157]:
!pip install pandas --upgrade

Requirement already up-to-date: pandas in /Users/dwightchurchill/.local/share/virtualenvs/dwightchurchill-Bl87FlYt/lib/python3.7/site-packages (0.25.0)


In [158]:
!pip install numpy --upgrade

Requirement already up-to-date: numpy in /Users/dwightchurchill/.local/share/virtualenvs/dwightchurchill-Bl87FlYt/lib/python3.7/site-packages (1.17.0)


In [164]:
!pip install datetime --upgrade

Requirement already up-to-date: datetime in /Users/dwightchurchill/.local/share/virtualenvs/dwightchurchill-Bl87FlYt/lib/python3.7/site-packages (4.3)


In [6]:
import datetime

In [79]:
import functools

In [7]:
import pandas as pd
import numpy as np
from pandas_datareader import data, wb
import ta as ta
import warnings
import sys, os



pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


## Data Wrangling

In [159]:
UST = 'https://raw.githubusercontent.com/black-swan-2/Black-Swan-2-Models/master/Data/USTREASURY-YIELD.csv'
FRED_PERMIT = 'https://raw.githubusercontent.com/black-swan-2/Black-Swan-2-Models/master/Data/FRED-PERMIT.csv'
ISMVS_GDP = 'https://raw.githubusercontent.com/black-swan-2/Black-Swan-2-Models/master/Data/ismvsgdp.csv'
USLGI = 'https://raw.githubusercontent.com/black-swan-2/Black-Swan-2-Models/master/Data/USLgI.csv'
INVERSION = 'https://raw.githubusercontent.com/black-swan-2/Black-Swan-2-Models/master/Data/inversion.csv'


In [207]:
def get_data():
    """
    Downloads and wrangles data for BlackSwan 2.0 VIX modeling. 
  
    Combines the two index datasets and creates a 
    new DataFrame (df) that contains the target feature (Trading Days with 3 
    STDV shifts based on a rolling window)
  
    :input: vix  => CBOE VIX Historical Data
    :input: gspc => GSPC S&P 500 index with matching start date to VIX
  
    :return: df  => cominbation VIX/GSPC Dataset with Target Classifier 
    """
  
    ############### Getting Data ##################
  
    vix = (data.DataReader('^VIX', 
                           "yahoo", 
                           start='1990-1-02', 
                           end='2019-5-31')
           .drop(columns = ['Volume', 'Adj Close']))
  
    gspc = data.DataReader('^GSPC', 
                           "yahoo", 
                           start='1990-1-02',
                           end='2019-5-31')
    
    treasury = (pd.read_csv(UST)
                .sort_values(by = 'Date')
                .drop(columns=['1 MO', '2 MO', '20 YR']))
    
    fred = pd.read_csv(FRED_PERMIT)
    
    ism = pd.read_csv(ISMVS_GDP).drop(index = [0,1])
    
    lag = pd.read_csv(USLGI)
    
    inv = (pd.read_csv(INVERSION)
           [['Date','3m1s_inversion','3m2s_inversion',
             '2s10s_inversion','2s30s_inversion']])
  
    ############### Wrangling Data #################
    
    # Rename the Columns
    vix.columns      = ['vix_high', 'vix_low', 'vix_open', 'vix_close']
    gspc.columns     = ['gspc_high', 'gspc_low', 'gspc_open',
                        'gspc_close','gspc_volume','gspc_adj_close']
  
    # Join the VIX and GSPC
    df = vix.join(gspc)
  
    # Pull Date columns out of the index
    df = df.reset_index()
    
    # Merge DF with the Treasury Data on the Date Feature
    # Date needs to be converted to Datetime format to match df['Date']
    treasury['Date'] = pd.to_datetime(treasury['Date'],
                                      infer_datetime_format=True)
    
    df = pd.merge(df, treasury, how='inner', on='Date')
    
    # Merge DF with the Inversion Features Engineered by Damerei
    # Sort and create key for merging
    inv = inv[inv['Date'] < '2019-06-01'].sort_values(by='Date')
    inv['Date'] = pd.to_datetime(inv['Date'], infer_datetime_format=True)
    
    df = pd.merge(df, inv, how='inner', on='Date')
    
    # A new feature is needed to act as a key for the following features
    df['y/m'] = df['Date'].map(lambda x: x.strftime('%Y-%m'))
    
    # Line up the Date column with the new key for the FRED Data
    fred = fred[fred['Date'] > '1989-12-01'].sort_values(by='Date')
    fred['Date'] = pd.to_datetime(fred['Date'], infer_datetime_format=True)
    fred['Date'] =  fred['Date'].map(lambda x: x.strftime('%Y-%m'))
    
    # Line up the Date column iwth the new key for the ISM/GDP Data
    ism = ism[ism['ticker'] > '1989-12-01'].sort_values(by='ticker')
    ism['ticker'] = pd.to_datetime(ism['ticker'], infer_datetime_format=True)
    ism['ticker'] =  ism['ticker'].map(lambda x: x.strftime('%Y-%m'))
    # Add most recent ISM data
    ism = (ism.append(pd.DataFrame([['2019-04', 52.8, np.nan]
                                   ,['2019-05', 52.1,np.nan]]
                                   ,columns=ism.columns))
           .reset_index()
           .drop(columns='index'))
    #Add most recent GDP Value
    ism['GDP CURY Index'].iloc[351] = 5.0
    # Fill in Quarterly GDP values
    ism = ism.fillna(method='ffill').fillna(method='bfill')
    
    # Wrangle the date column for the Lagging Index Dataframe
    lag['Date'] = pd.to_datetime(lag['Date'], format='%b-%y')
    #Create a pivot point around 2019 to accomodate the 2-digit year format
    for i in np.arange(len(lag)):
        if lag['Date'][i].year > 2019:
            lag['Date'][i] = lag['Date'][i].replace(year=lag['Date'][i].year-100)
        else:
            pass
    # Map to the y/m key format
    lag['Date'] =  lag['Date'].map(lambda x: x.strftime('%Y-%m'))
    #Slice the dataframe to relevant time period
    lag = lag[lag['Date'] > '1989-12']
    

    # Create New Columns with a single iteration using list comprehension
    f  = []
    m  = []
    g  = []
    ll = []
    lg = []

    for i in np.arange(len(df)):
        # Merge the New Private Housing Units Authorized by Building Permits (FRED)
        f.append(float(fred.loc[fred['Date'] == df['y/m'][i]]['Value']
                       .values
                       .tolist()[0]))
        
        # Merge the ISM NAPMPMI Index
        m.append(float(ism.loc[ism['ticker'] == df['y/m'][i]]['NAPMPMI Index']
                       .values
                       .tolist()[0]))
        
        # Merge the GDP CURY Index
        g.append(float(ism.loc[ism['ticker'] == df['y/m'][i]]['GDP CURY Index']
                       .values
                       .tolist()[0]))
        
            # Merge the Lagging Index Level Column
        ll.append(float(lag.loc[lag['Date'] == df['y/m'][i]]['Level']
                        .values
                        .tolist()[0]))
        
        # Merge the Lagging Index Growth Column
        lg.append(float(lag.loc[lag['Date'] == df['y/m'][i]]['Growth']
                        .values
                        .tolist()[0]))
    
    df['lag_index_level'] = ll
    df['lag_index_growth'] = lg
    df['fred'] = f
    df['ism'] = m
    df['gdp_cury'] = g
    
  
    ############### Momemntum Feature Engineering ################
  
    # Awesome Oscillator
    df['mom_ao']=ta.momentum.ao(df['gspc_high'],
                                df['gspc_low'],
                                s=5,len=34,
                                fillna=True)

    # Money Flow Index
    df['mom_mf']=ta.momentum.money_flow_index(df['gspc_high'],
                                              df['gspc_low'],
                                              df['gspc_close'],
                                              df['gspc_volume'],
                                              n=14,fillna=True)
  
    # Relative Strength Index
    df['mom_rsi'] = ta.momentum.rsi(df['gspc_close'],
                                    n=14,
                                    fillna=True)
  
    # Stochasitc Oscillator
    df['mom_stoch']=ta.momentum.stoch(df['gspc_high'],
                                      df['gspc_low'],
                                      df['gspc_close'],
                                      n=14,
                                      fillna=True)
  
    # Stochasitc Signal
    df['mom_st_sig']=ta.momentum.stoch_signal(df['gspc_high'],
                                              df['gspc_low'],
                                              df['gspc_close'],
                                              n=14,
                                              d_n=3,
                                              fillna=True)
  
    # True Strength Indicator
    df['mom_tsi'] = ta.momentum.tsi(df['gspc_close'],
                                    r=25,
                                    s=13,
                                    fillna=True)
  
    # Ultimate Oscillator
    df['mom_uo'] = ta.momentum.uo(df['gspc_high'],
                                  df['gspc_low'],
                                  df['gspc_close'], 
                                  s=7, 
                                  m=14, 
                                  len=28, 
                                  ws=4.0, 
                                  wm=2.0, 
                                  wl=1.0,
                                  fillna=True)
  
    # Williams %R
    df['mom_wr']=ta.momentum.wr(df['gspc_high'],
                                df['gspc_low'],
                                df['gspc_close'],
                                lbp=14,fillna=True)
  
    ############### Volume Feature Engineering ####################
  
    # Accumulation/Distribution Index
    df['vol_adi']=ta.volume.acc_dist_index(df['gspc_high'],
                                           df['gspc_low'],
                                           df['gspc_close'],
                                           df['gspc_volume'],
                                           fillna=True)
  
    # Chaikin Money Flow
    df['vol_cmf'] = ta.volume.chaikin_money_flow(df['gspc_high'],
                                                 df['gspc_low'],
                                                 df['gspc_close'],
                                                 df['gspc_volume'],
                                                 n=20,fillna=True)
  
    # Ease of Movement
    df['vol_eom'] = ta.volume.ease_of_movement(df['gspc_high'],
                                               df['gspc_low'],
                                               df['gspc_close'],
                                               df['gspc_volume'],
                                               n=20,fillna=True)
  
    # Force Index
    df['vol_fm'] = ta.volume.force_index(df['gspc_close'],
                                         df['gspc_volume'],
                                         n=2,fillna=True)
  
    # Negative Volume Index
    df['vol_nvi'] = ta.volume.negative_volume_index(df['gspc_close'],
                                                    df['gspc_volume'],
                                                    fillna=True)
  
    # On-Balance Volume
    df['vol_obv'] = ta.volume.on_balance_volume(df['gspc_close'],
                                                df['gspc_volume'],
                                                fillna=True)
  
    # Volume-Price Trend
    df['vol_vpt'] = ta.volume.volume_price_trend(df['gspc_close'],
                                                 df['gspc_volume'],
                                                 fillna=True)
    
    ############### Volatility Feature Engineering
    
    #Average True Range
    df['atr_low'] = ta.volatility.average_true_range(df['gspc_high'],
                                                     df['gspc_low'],
                                                     df['gspc_close'],
                                                     n=23)
    
    df['atr_high'] = ta.volatility.average_true_range(df['gspc_high'],
                                                      df['gspc_low'],
                                                      df['gspc_close'],
                                                      n=37)

    ############### Target Creation #################
    
    # List Comprehension to create a Percent Moved 
    p = []

    for i in np.arange(len(df)):
                # Day 2 Close   subtracted by  Day 1 Close   div by Day 1 Close  * 100
        p.append(((df['gspc_close'].shift(1)[i] - df['gspc_close'][i]) / df['gspc_close'][i])*100)
    
    df['percent_move'] = p
    
    # Create a stdev column based on the VIX
    df['stdev'] = ((df['vix_close']/np.sqrt(256))).rolling(252).mean()
    
    df['percent_move'] = np.abs(df['percent_move']).shift(1)
    
    ########################### TARGET FEATURE #############################################
    df['3_sigma_event'] = np.where(np.abs(df['percent_move']) > 3 * df['stdev'], 1, 0)
    ########################### TARGET FEATURE #############################################
    
  
     # Determine daily market movement between Close and Close
    df['vix_move']  = (1 - df['vix_close']
                       .shift(1)/df['vix_close'])
  
    df['gspc_move'] = (1 - df['gspc_close']
                        .shift(1)/df['gspc_close'])
  
    ############## Handling Null Values ##################
    
    # Interpolating the Null Values for 30yr Treasury Bonds
    df['30 YR'] = (df['30 YR'].interpolate(method='spline',
                                           order=4))

    # Drop the rest
    df = df.dropna()
    
    # Reset the index for mistake free slicing
    df = df.reset_index().drop(columns = 'index')
  
    return df
  

In [208]:
df = get_data()

In [209]:
df.head()

,Date,vix_high,vix_low,vix_open,vix_close,gspc_high,gspc_low,gspc_open,gspc_close,gspc_volume,gspc_adj_close,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,30 YR,3m1s_inversion,3m2s_inversion,2s10s_inversion,2s30s_inversion,y/m,lag_index_level,lag_index_growth,fred,ism,gdp_cury,mom_ao,mom_mf,mom_rsi,mom_stoch,mom_st_sig,mom_tsi,mom_uo,mom_wr,vol_adi,vol_cmf,vol_eom,vol_fm,vol_nvi,vol_obv,vol_vpt,atr_low,atr_high,percent_move,stdev,3_sigma_event,vix_move,gspc_move
0,1991-01-03,27.930000,27.930000,27.930000,27.930000,326.529999,321.899994,326.459991,321.910004,141450000,321.910004,6.64,6.71,6.72,7.08,7.27,7.56,7.86,7.93,8.11,False,False,False,False,1991-01,104.1,2.1,786.0,39.2,4.5,3.569002,46.537378,30.616455,0.094700,32.283007,8.595531,38.908782,-99.905300,-2.671184e+08,0.129211,-6.297124e-09,-2.270291e+08,804.790977,2.401960e+09,-3.408863e+06,3.760180,4.133586,1.154844,1.440880,0,0.046903,-0.014103
1,1991-01-04,27.190001,27.190001,27.190001,27.190001,322.350006,318.869995,321.910004,321.000000,140820000,321.000000,6.73,6.82,6.83,7.17,7.37,7.65,7.94,8.02,8.20,False,False,False,False,1991-01,104.1,2.1,786.0,39.2,4.5,1.730324,47.151724,28.796672,15.661794,13.533556,7.840947,42.658298,-84.338206,-1.092754e+08,0.073116,-1.429087e-08,-7.924318e+07,802.515923,2.261140e+09,-2.365257e+06,3.747999,4.115922,1.410335,1.443348,0,-0.027216,-0.002835
2,1991-01-07,28.950001,28.950001,28.950001,28.950001,320.970001,315.440002,320.970001,315.440002,130610000,315.440002,6.71,6.84,6.84,7.20,7.43,7.75,8.04,8.13,8.32,False,False,False,False,1991-01,104.1,2.1,786.0,39.2,4.5,-0.222910,47.759075,20.293223,0.000000,5.252164,6.517319,35.161492,-100.000000,-9.904705e+07,0.098190,-2.308569e-08,7.013481e+07,788.615653,2.130530e+09,-2.660361e+06,3.826782,4.154951,0.283490,1.446017,0,0.060794,-0.017626
3,1991-01-08,30.379999,30.379999,30.379999,30.379999,316.970001,313.790009,315.440002,314.899994,143390000,314.899994,6.64,6.74,6.75,7.15,7.39,7.74,8.06,8.16,8.37,False,False,False,False,1991-01,104.1,2.1,786.0,39.2,4.5,-2.875352,47.255924,19.643185,5.942108,7.201301,5.262544,36.594105,-94.057892,-1.738986e+08,0.089016,-2.323096e-08,-1.567702e+07,788.615653,1.987140e+09,-2.507751e+06,3.798660,4.128600,1.762617,1.448785,0,0.047070,-0.001715
4,1991-01-09,33.299999,33.299999,33.299999,33.299999,320.730011,310.929993,314.899994,311.489990,191100000,311.489990,6.44,6.61,6.68,7.10,7.46,7.81,8.12,8.25,8.46,False,False,False,False,1991-01,104.1,2.1,786.0,39.2,4.5,-5.357588,37.837654,15.926122,2.599802,2.847303,3.743329,26.979367,-97.400198,-2.125488e+08,-0.017458,-2.174151e-08,-2.389362e+08,788.615653,1.796040e+09,-2.314865e+06,4.059589,4.281882,0.171486,1.452056,0,0.087688,-0.010947


In [210]:
start_date = datetime.datetime(2008, 12, 31)

df = df[df['Date'] > start_date]

In [211]:
df.head()

,Date,vix_high,vix_low,vix_open,vix_close,gspc_high,gspc_low,gspc_open,gspc_close,gspc_volume,gspc_adj_close,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,30 YR,3m1s_inversion,3m2s_inversion,2s10s_inversion,2s30s_inversion,y/m,lag_index_level,lag_index_growth,fred,ism,gdp_cury,mom_ao,mom_mf,mom_rsi,mom_stoch,mom_st_sig,mom_tsi,mom_uo,mom_wr,vol_adi,vol_cmf,vol_eom,vol_fm,vol_nvi,vol_obv,vol_vpt,atr_low,atr_high,percent_move,stdev,3_sigma_event,vix_move,gspc_move
4494,2009-01-02,39.820000,36.880001,39.580002,39.189999,934.729980,899.349976,902.989990,931.799988,4048270000,931.799988,0.08,0.28,0.40,0.88,1.14,1.72,2.07,2.46,2.83,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,19.188677,54.373563,70.641345,96.485977,77.194122,-9.042839,68.861605,-3.514023,4.693293e+09,0.090444,1.908986e-08,1.730653e+10,922.503526,2.932508e+11,1.870399e+08,36.975585,39.464797,1.396068,2.032240,0,-0.020669,0.030640
4495,2009-01-05,40.220001,38.299999,39.240002,39.080002,936.630005,919.530029,929.169983,927.450012,5413910000,927.450012,0.14,0.32,0.43,0.78,1.08,1.67,2.07,2.49,3.00,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,28.948385,45.179753,67.842757,88.461547,87.278816,-8.439898,70.776905,-11.538453,2.978832e+09,0.113486,1.931402e-08,3.003149e+10,922.503526,2.878369e+11,1.026839e+08,36.111428,38.860343,3.063961,2.036186,0,-0.002815,-0.004690
4496,2009-01-06,39.330002,37.340000,38.060001,38.560001,943.849976,927.280029,931.169983,934.700012,5392620000,934.700012,0.14,0.31,0.45,0.80,1.10,1.68,2.07,2.51,3.04,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,41.743648,53.586917,70.119271,89.456134,91.467886,-7.810315,68.329414,-10.543866,-9.619383e+08,0.052175,2.327680e-08,3.898648e+09,929.714859,2.932296e+11,1.688076e+07,35.261798,38.257900,0.469025,2.040171,0,-0.013485,0.007756
4497,2009-01-07,43.820000,40.119999,40.290001,43.389999,927.450012,902.369995,927.450012,906.650024,4704940000,906.650024,0.11,0.29,0.44,0.82,1.15,1.66,2.02,2.52,3.05,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,47.106382,44.854041,53.280736,57.133021,78.350234,-7.486553,57.640913,-42.866979,-3.662112e+09,-0.013996,3.620205e-09,1.474657e+10,901.814474,2.885246e+11,-9.903861e+07,35.134330,38.097687,0.775650,2.044995,0,0.111316,-0.030938
4498,2009-01-08,44.599998,42.560001,43.380001,42.560001,910.000000,896.809998,905.729980,909.729980,4991550000,909.729980,0.09,0.28,0.44,0.83,1.16,1.60,1.95,2.47,3.04,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,46.110883,53.877409,54.660187,60.682176,69.090444,-7.156613,59.267594,-39.317824,1.688083e+09,0.083738,2.051310e-09,1.001473e+10,901.814474,2.935162e+11,-1.242368e+08,34.180228,37.424506,3.093805,2.049650,0,-0.019502,0.003386


In [212]:
print(df.shape,'\n', df.isnull().sum(),'\n',
      df.info(),'\n', df.describe(),'\n\n')
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2601 entries, 4494 to 7094
Data columns (total 52 columns):
Date                2601 non-null datetime64[ns]
vix_high            2601 non-null float64
vix_low             2601 non-null float64
vix_open            2601 non-null float64
vix_close           2601 non-null float64
gspc_high           2601 non-null float64
gspc_low            2601 non-null float64
gspc_open           2601 non-null float64
gspc_close          2601 non-null float64
gspc_volume         2601 non-null int64
gspc_adj_close      2601 non-null float64
3 MO                2601 non-null float64
6 MO                2601 non-null float64
1 YR                2601 non-null float64
2 YR                2601 non-null float64
3 YR                2601 non-null float64
5 YR                2601 non-null float64
7 YR                2601 non-null float64
10 YR               2601 non-null float64
30 YR               2601 non-null float64
3m1s_inversion      2601 non-null bool
3m2s_i

,Date,vix_high,vix_low,vix_open,vix_close,gspc_high,gspc_low,gspc_open,gspc_close,gspc_volume,gspc_adj_close,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,30 YR,3m1s_inversion,3m2s_inversion,2s10s_inversion,2s30s_inversion,y/m,lag_index_level,lag_index_growth,fred,ism,gdp_cury,mom_ao,mom_mf,mom_rsi,mom_stoch,mom_st_sig,mom_tsi,mom_uo,mom_wr,vol_adi,vol_cmf,vol_eom,vol_fm,vol_nvi,vol_obv,vol_vpt,atr_low,atr_high,percent_move,stdev,3_sigma_event,vix_move,gspc_move
4494,2009-01-02,39.820000,36.880001,39.580002,39.189999,934.729980,899.349976,902.989990,931.799988,4048270000,931.799988,0.08,0.28,0.40,0.88,1.14,1.72,2.07,2.46,2.83,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,19.188677,54.373563,70.641345,96.485977,77.194122,-9.042839,68.861605,-3.514023,4.693293e+09,0.090444,1.908986e-08,1.730653e+10,922.503526,2.932508e+11,1.870399e+08,36.975585,39.464797,1.396068,2.032240,0,-0.020669,0.030640
4495,2009-01-05,40.220001,38.299999,39.240002,39.080002,936.630005,919.530029,929.169983,927.450012,5413910000,927.450012,0.14,0.32,0.43,0.78,1.08,1.67,2.07,2.49,3.00,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,28.948385,45.179753,67.842757,88.461547,87.278816,-8.439898,70.776905,-11.538453,2.978832e+09,0.113486,1.931402e-08,3.003149e+10,922.503526,2.878369e+11,1.026839e+08,36.111428,38.860343,3.063961,2.036186,0,-0.002815,-0.004690
4496,2009-01-06,39.330002,37.340000,38.060001,38.560001,943.849976,927.280029,931.169983,934.700012,5392620000,934.700012,0.14,0.31,0.45,0.80,1.10,1.68,2.07,2.51,3.04,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,41.743648,53.586917,70.119271,89.456134,91.467886,-7.810315,68.329414,-10.543866,-9.619383e+08,0.052175,2.327680e-08,3.898648e+09,929.714859,2.932296e+11,1.688076e+07,35.261798,38.257900,0.469025,2.040171,0,-0.013485,0.007756
4497,2009-01-07,43.820000,40.119999,40.290001,43.389999,927.450012,902.369995,927.450012,906.650024,4704940000,906.650024,0.11,0.29,0.44,0.82,1.15,1.66,2.02,2.52,3.05,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,47.106382,44.854041,53.280736,57.133021,78.350234,-7.486553,57.640913,-42.866979,-3.662112e+09,-0.013996,3.620205e-09,1.474657e+10,901.814474,2.885246e+11,-9.903861e+07,35.134330,38.097687,0.775650,2.044995,0,0.111316,-0.030938
4498,2009-01-08,44.599998,42.560001,43.380001,42.560001,910.000000,896.809998,905.729980,909.729980,4991550000,909.729980,0.09,0.28,0.44,0.83,1.16,1.60,1.95,2.47,3.04,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,46.110883,53.877409,54.660187,60.682176,69.090444,-7.156613,59.267594,-39.317824,1.688083e+09,0.083738,2.051310e-09,1.001473e+10,901.814474,2.935162e+11,-1.242368e+08,34.180228,37.424506,3.093805,2.049650,0,-0.019502,0.003386


### Companies' Date ###

In [213]:
Amazon = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Amazon.csv'
Apple = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Apple.csv'
BOFA = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/BOFA.csv'
Berkshire = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Berkshire.csv'
Google = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Google.csv'
JPM = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/JPM.csv'
Johnson = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Johnson.csv'
Mastercard = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Mastercard.csv'
Microsoft = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Microsoft.csv'
Proctor = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Proctor.csv'
Visa = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/VISA.csv'


In [214]:
Amazon = pd.read_csv(Amazon)
Apple = pd.read_csv(Apple)
BOFA = pd.read_csv(BOFA)
Berkshire = pd.read_csv(Berkshire)
Google = pd.read_csv(Google)
JPM = pd.read_csv(JPM)
Johnson = pd.read_csv(Johnson)
Mastercard = pd.read_csv(Mastercard)
Microsoft = pd.read_csv(Microsoft)
Proctor = pd.read_csv(Proctor)
Visa = pd.read_csv(Visa)

In [215]:
Amazon['date'] = pd.to_datetime(Amazon['date'], yearfirst = True)
Apple['date'] = pd.to_datetime(Apple['date'], yearfirst = True)
BOFA['date'] = pd.to_datetime(BOFA['date'], yearfirst = True)
Berkshire['date'] = pd.to_datetime(Berkshire['date'], yearfirst = True)
Google['date'] = pd.to_datetime(Google['date'], yearfirst = True)
JPM['date'] = pd.to_datetime(JPM['date'], yearfirst = True)
Johnson['date'] = pd.to_datetime(Johnson['date'], yearfirst = True)
Mastercard['date'] = pd.to_datetime(Mastercard['date'], yearfirst = True)
Microsoft['date'] = pd.to_datetime(Microsoft['date'], yearfirst = True)
Proctor['date'] = pd.to_datetime(Proctor['date'], yearfirst = True)
Visa['date'] = pd.to_datetime(Visa['date'], yearfirst = True)


Amazon.head()

,date,Close,Volume
0,2009-01-02,54.36,7296400
1,2009-01-05,54.06,9509800
2,2009-01-06,57.36,11080100
3,2009-01-07,56.20,7942700
4,2009-01-08,57.16,6577900


In [216]:
Boeing = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/ba.csv'
Citi = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/c.csv'
Comcast = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/cmcsa.csv'
Cisco = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/csco.csv'
Disney = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/dis.csv'
HD = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/hd.csv'
Intel = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/intc.csv'
Coke = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/ko.csv'
Mcdonalds = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/mcd.csv'
Merck = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/mrk.csv'
Pepsi = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/pep.csv'
Pfizer = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/pfe.csv'
ATT = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/t.csv'
UNH = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/unh.csv'
Verizon = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/vz.csv'
Wells = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/wfc.csv'
Walmart = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/wmt.csv'

In [217]:
my_companies = [Boeing, Citi, Comcast, Cisco, Disney, HD, Intel, Coke, Mcdonalds, Merck, Pepsi, Pfizer, ATT, UNH, Verizon, Wells, Walmart]


In [218]:
Boeing = pd.read_csv(Boeing)
Citi = pd.read_csv(Citi)
Comcast = pd.read_csv(Comcast)
Cisco = pd.read_csv(Cisco)
Disney = pd.read_csv(Disney)
HD = pd.read_csv(HD)
Intel = pd.read_csv(Intel)
Coke = pd.read_csv(Coke)
Mcdonalds = pd.read_csv(Mcdonalds)
Merck = pd.read_csv(Merck)
Pepsi = pd.read_csv(Pepsi)
Pfizer = pd.read_csv(Pfizer)
ATT = pd.read_csv(ATT)
UNH = pd.read_csv(UNH)
Verizon = pd.read_csv(Verizon)
Wells = pd.read_csv(Wells)
Walmart = pd.read_csv(Walmart)

Boeing.head(2)


,Unnamed: 0,date,Close,Volume
0,2767,2009-01-02,45.25,7010200.0
1,2768,2009-01-05,46.17,6917400.0


In [219]:
companies = [Amazon, Apple, BOFA, Berkshire, Google, JPM, Johnson, Mastercard, Microsoft, Proctor, Visa, Boeing, Citi, Comcast, 
            Cisco, Disney, HD, Intel, Coke, Mcdonalds, Merck, Pepsi, Pfizer, ATT, UNH, Verizon, Wells, Walmart, Boeing]

In [220]:
df = df.rename(columns = {'Date': 'date'})

In [221]:
df['date'] = pd.to_datetime(df['date'])

df['date'].dtypes

dtype('<M8[ns]')

In [222]:
df.head()

,date,vix_high,vix_low,vix_open,vix_close,gspc_high,gspc_low,gspc_open,gspc_close,gspc_volume,gspc_adj_close,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,30 YR,3m1s_inversion,3m2s_inversion,2s10s_inversion,2s30s_inversion,y/m,lag_index_level,lag_index_growth,fred,ism,gdp_cury,mom_ao,mom_mf,mom_rsi,mom_stoch,mom_st_sig,mom_tsi,mom_uo,mom_wr,vol_adi,vol_cmf,vol_eom,vol_fm,vol_nvi,vol_obv,vol_vpt,atr_low,atr_high,percent_move,stdev,3_sigma_event,vix_move,gspc_move
4494,2009-01-02,39.820000,36.880001,39.580002,39.189999,934.729980,899.349976,902.989990,931.799988,4048270000,931.799988,0.08,0.28,0.40,0.88,1.14,1.72,2.07,2.46,2.83,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,19.188677,54.373563,70.641345,96.485977,77.194122,-9.042839,68.861605,-3.514023,4.693293e+09,0.090444,1.908986e-08,1.730653e+10,922.503526,2.932508e+11,1.870399e+08,36.975585,39.464797,1.396068,2.032240,0,-0.020669,0.030640
4495,2009-01-05,40.220001,38.299999,39.240002,39.080002,936.630005,919.530029,929.169983,927.450012,5413910000,927.450012,0.14,0.32,0.43,0.78,1.08,1.67,2.07,2.49,3.00,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,28.948385,45.179753,67.842757,88.461547,87.278816,-8.439898,70.776905,-11.538453,2.978832e+09,0.113486,1.931402e-08,3.003149e+10,922.503526,2.878369e+11,1.026839e+08,36.111428,38.860343,3.063961,2.036186,0,-0.002815,-0.004690
4496,2009-01-06,39.330002,37.340000,38.060001,38.560001,943.849976,927.280029,931.169983,934.700012,5392620000,934.700012,0.14,0.31,0.45,0.80,1.10,1.68,2.07,2.51,3.04,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,41.743648,53.586917,70.119271,89.456134,91.467886,-7.810315,68.329414,-10.543866,-9.619383e+08,0.052175,2.327680e-08,3.898648e+09,929.714859,2.932296e+11,1.688076e+07,35.261798,38.257900,0.469025,2.040171,0,-0.013485,0.007756
4497,2009-01-07,43.820000,40.119999,40.290001,43.389999,927.450012,902.369995,927.450012,906.650024,4704940000,906.650024,0.11,0.29,0.44,0.82,1.15,1.66,2.02,2.52,3.05,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,47.106382,44.854041,53.280736,57.133021,78.350234,-7.486553,57.640913,-42.866979,-3.662112e+09,-0.013996,3.620205e-09,1.474657e+10,901.814474,2.885246e+11,-9.903861e+07,35.134330,38.097687,0.775650,2.044995,0,0.111316,-0.030938
4498,2009-01-08,44.599998,42.560001,43.380001,42.560001,910.000000,896.809998,905.729980,909.729980,4991550000,909.729980,0.09,0.28,0.44,0.83,1.16,1.60,1.95,2.47,3.04,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,46.110883,53.877409,54.660187,60.682176,69.090444,-7.156613,59.267594,-39.317824,1.688083e+09,0.083738,2.051310e-09,1.001473e+10,901.814474,2.935162e+11,-1.242368e+08,34.180228,37.424506,3.093805,2.049650,0,-0.019502,0.003386


In [236]:
company_dict['amazon'].tail()

,date,Close,Volume
2654,2019-07-22,1985.63,2908111
2655,2019-07-23,1994.49,2703480
2656,2019-07-24,2000.81,2631300
2657,2019-07-25,1973.82,4136394
2658,2019-07-26,1943.05,4919870


### Merging Data ###

In [224]:
company_list = ['amazon', 'apple', 'bofa', 'berkshire', 'google', 'jpm', 'johnson', 'mastercard', 
                'microsoft', 'proctor', 'visa', 'boeing', 'citi', 'comcast', 'cisco', 'disney', 
                'hd', 'intel', 'coke', 'mcdonalds', 'merck', 'pepsi', 'pfizer', 'att', 'unh', 
                'verizon', 'wells', 'walmart', 'boeing']

In [225]:
for c in companies: 
    c['date'] = c['date'].astype(str)

df['date'] = df['date'].astype(str)

In [226]:
companies[0]['date'].dtype

dtype('O')

In [227]:
df['date'].dtype

dtype('O')

In [243]:
company_dict = dict.fromkeys(company_list)
company_dict = dict(zip(company_list, companies))

In [244]:
mergedCompanies = {}

for k, v in company_dict.items():
    c1 = {k : pd.merge(df, v, how="inner", on="date")}
    mergedCompanies.update(c1)

In [245]:
for k, v in mergedCompanies.items():
    v['date'] = pd.to_datetime(v['date'])

SyntaxError: invalid syntax (<ipython-input-245-2335dcf0530a>, line 4)

In [242]:
mergedCompanies['amazon']['date'].head()

0   2009-01-02
1   2009-01-05
2   2009-01-06
3   2009-01-07
4   2009-01-08
Name: date, dtype: datetime64[ns]

## Target and Features

>**Targets** I have created a number of target columns in order to create a really rough gradient descent for rolling Standard Deviations of day-day GSPC closes. 

>**Leakage** These are features to be dropped as they are deemed to be leaky or noisy

>**features** This is the resulting dataframe to be inserted into a model. 


In [0]:
target  = 'Close'

#to_drop  = ['date','y/m','percent_move','mom_wr','stdev','gspc_adj_close' , '3_sigma_event']

#features  = (df.drop(columns = target)
#            .drop(columns = to_drop)
#            .columns)

In [0]:
features

Index(['vix_high', 'vix_low', 'vix_open', 'vix_close', 'gspc_high', 'gspc_low',
       'gspc_open', 'gspc_close', 'gspc_volume', '3 MO', '6 MO', '1 YR',
       '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '30 YR', '3m1s_inversion',
       '3m2s_inversion', '2s10s_inversion', '2s30s_inversion',
       'lag_index_level', 'lag_index_growth', 'fred', 'ism', 'gdp_cury',
       'mom_ao', 'mom_mf', 'mom_rsi', 'mom_stoch', 'mom_st_sig', 'mom_tsi',
       'mom_uo', 'vol_adi', 'vol_cmf', 'vol_eom', 'vol_fm', 'vol_nvi',
       'vol_obv', 'vol_vpt', 'atr_low', 'atr_high', '3_sigma_event',
       'vix_move', 'gspc_move', 'Volume'],
      dtype='object')

In [0]:
Amazon.head()

,date,vix_high_x,vix_low_x,vix_open_x,vix_close_x,gspc_high_x,gspc_low_x,gspc_open_x,gspc_close_x,gspc_volume_x,gspc_adj_close_x,3 MO_x,6 MO_x,1 YR_x,2 YR_x,3 YR_x,5 YR_x,7 YR_x,10 YR_x,30 YR_x,3m1s_inversion_x,3m2s_inversion_x,2s10s_inversion_x,2s30s_inversion_x,y/m_x,lag_index_level_x,lag_index_growth_x,fred_x,ism_x,gdp_cury_x,mom_ao_x,mom_mf_x,mom_rsi_x,mom_stoch_x,mom_st_sig_x,mom_tsi_x,mom_uo_x,mom_wr_x,vol_adi_x,vol_cmf_x,vol_eom_x,vol_fm_x,vol_nvi_x,vol_obv_x,vol_vpt_x,atr_low_x,atr_high_x,percent_move_x,stdev_x,3_sigma_event_x,vix_move_x,gspc_move_x,vix_high_y,vix_low_y,vix_open_y,vix_close_y,gspc_high_y,gspc_low_y,gspc_open_y,gspc_close_y,gspc_volume_y,gspc_adj_close_y,3 MO_y,6 MO_y,1 YR_y,2 YR_y,3 YR_y,5 YR_y,7 YR_y,10 YR_y,30 YR_y,3m1s_inversion_y,3m2s_inversion_y,2s10s_inversion_y,2s30s_inversion_y,y/m_y,lag_index_level_y,lag_index_growth_y,fred_y,ism_y,gdp_cury_y,mom_ao_y,mom_mf_y,mom_rsi_y,mom_stoch_y,mom_st_sig_y,mom_tsi_y,mom_uo_y,mom_wr_y,vol_adi_y,vol_cmf_y,vol_eom_y,vol_fm_y,vol_nvi_y,vol_obv_y,vol_vpt_y,atr_low_y,atr_high_y,percent_move_y,stdev_y,3_sigma_event_y,vix_move_y,gspc_move_y,Close,Volume
0,2009-01-02,39.820000,36.880001,39.580002,39.189999,934.729980,899.349976,902.989990,931.799988,4048270000,931.799988,0.08,0.28,0.40,0.88,1.14,1.72,2.07,2.46,2.83,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,19.188677,54.373563,70.641345,96.485977,77.194122,-9.042839,68.861605,-3.514023,4.693293e+09,0.090444,1.908986e-08,1.730653e+10,922.503526,2.932508e+11,1.870399e+08,36.975585,39.464797,1.396068,2.032240,0,-0.020669,0.030640,39.820000,36.880001,39.580002,39.189999,934.729980,899.349976,902.989990,931.799988,4048270000,931.799988,0.08,0.28,0.40,0.88,1.14,1.72,2.07,2.46,2.83,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,19.188677,54.373563,70.641345,96.485977,77.194122,-9.042839,68.861605,-3.514023,4.693293e+09,0.090444,1.908986e-08,1.730653e+10,922.503526,2.932508e+11,1.870399e+08,36.975585,39.464797,1.396068,2.032240,0,-0.020669,0.030640,54.36,7296400
1,2009-01-05,40.220001,38.299999,39.240002,39.080002,936.630005,919.530029,929.169983,927.450012,5413910000,927.450012,0.14,0.32,0.43,0.78,1.08,1.67,2.07,2.49,3.00,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,28.948385,45.179753,67.842757,88.461547,87.278816,-8.439898,70.776905,-11.538453,2.978832e+09,0.113486,1.931402e-08,3.003149e+10,922.503526,2.878369e+11,1.026839e+08,36.111428,38.860343,3.063961,2.036186,0,-0.002815,-0.004690,40.220001,38.299999,39.240002,39.080002,936.630005,919.530029,929.169983,927.450012,5413910000,927.450012,0.14,0.32,0.43,0.78,1.08,1.67,2.07,2.49,3.00,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,28.948385,45.179753,67.842757,88.461547,87.278816,-8.439898,70.776905,-11.538453,2.978832e+09,0.113486,1.931402e-08,3.003149e+10,922.503526,2.878369e+11,1.026839e+08,36.111428,38.860343,3.063961,2.036186,0,-0.002815,-0.004690,54.06,9509800
2,2009-01-06,39.330002,37.340000,38.060001,38.560001,943.849976,927.280029,931.169983,934.700012,5392620000,934.700012,0.14,0.31,0.45,0.80,1.10,1.68,2.07,2.51,3.04,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,41.743648,53.586917,70.119271,89.456134,91.467886,-7.810315,68.329414,-10.543866,-9.619383e+08,0.052175,2.327680e-08,3.898648e+09,929.714859,2.932296e+11,1.688076e+07,35.261798,38.257900,0.469025,2.040171,0,-0.013485,0.007756,39.330002,37.340000,38.060001,38.560001,943.849976,927.280029,931.169983,934.700012,5392620000,934.700012,0.14,0.31,0.45,0.80,1.10,1.68,2.07,2.51,3.04,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,41.743648,53.586917,70.119271,89.456134,91.467886,-7.810315,68.329414,-10.543866,-9.619383e+08,0.052175,2.327680e-08,3.898648e+09,929.714859,2.932296e+11,1.688076e+07,35.261798,38.257900,0.469025,2.040171,0,-0.013485,0.007756,57.36,11080100
3,2009-01-07,43.820000,40.119999,40.290001,43.389999,927.450012,902.369995,927.450012,906.650024,4704940000,906.650024,0.11,0.29,0.44,0.82,1.15,1.66,2.02,2.52,3.05,False,False,F

## Modified Regressor

In [0]:
# Model
from xgboost import XGBRegressor
# Regression metrics
from sklearn.metrics import mean_squared_error, r2_score
# Classification Metrics
from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix, accuracy_score

In [0]:
model   = XGBRegressor (base_score=0.72, booster='gbtree', colsample_bylevel=1,
                        colsample_bytree=1, gamma=0, learning_rate=0.13, max_delta_step=0,
                        max_depth=8, min_child_weight=.8, missing=None, n_estimators=180,
                        n_jobs=-1, objective='reg:squarederror',random_state=42, reg_alpha=0, 
                        reg_lambda=1, scale_pos_weight=1,seed=42, silent=True, subsample=1, 
                        eval_metric='rmse', normalize_type='forest')

In [0]:
def split(cut, features=features, df=df):
    """
    Temporal Train Test Split. Holdout is most recent data.  
    """
    
    # Creates X and y features. 
    # Shifts the target to a day later. 
    X = df[features]
    y = df['Close']
    
    # Train Test Split: 70% Train, 30% Test
    X_train = X[:int(len(X.index)*cut)]
    X_test= X[int(len(X.index)*cut):len(X.index)]
    y_train=y[:int(len(y.index)*cut)]
    y_test=y[int(len(y.index)*cut):len(y.index)]
    
    return X_train, X_test, y_train, y_test
    

In [0]:
def second_split(cut_2, X_train, X_test, y_train, y_test):
    """
    Splits Train-Test data into a smaller set for stretching
    gradient decent. 
    """
    frames = [X_train, X_test, y_train, y_test]
    
    for frame in frames:
        frame = frame.reset_index().drop(columns = 'index')
    
    # Train Test Split: 70% Train, 30% Test
    X_train_2 = X_train[:int(len(X_train.index)*cut_2)]
    X_test_2= X_train[int(len(X_train.index)*cut_2):len(X_train.index)]
    y_train_2=y_train[:int(len(y_train.index)*cut_2)]
    y_test_2=y_train[int(len(y_train.index)*cut_2):len(y_train.index)]
    
    return X_train_2, X_test_2, y_train_2, y_test_2
    

In [0]:
def bootstrap_gradient(metric, X_train, X_test, y_train, y_test,
                       X_train_2, X_test_2, y_train_2, y_test_2,
                       y_pred):
    
    # Grab the Standard Deviation and target data
    # slice it based on the training data
    stdevs = df.iloc[len(X_train_2):len(X_train)]['stdev'].values
    three_sigma = df.iloc[len(X_train_2):len(X_train)]['3_sigma_event'].values
    highest_score = 0
    raise_by = 0
    
    # Brute force a gradient descent
    
    for j in np.arange(start=1, stop=3, step=.001):   
        y_pred_mod = []
        for i in np.arange(len(y_pred)):    
            if y_pred[i] < y_test_2.values[i]:
                y_pred_mod.append(y_pred[i]**j)
            else:
                y_pred_mod.append(y_pred[i])
            
        # Build a class prediction
        modified_prediction = np.array(y_pred_mod)
        class_pred = (np.where(np.divide(modified_prediction, 
                                         stdevs) >= 3, 1, 0))
        score = metric(class_pred, three_sigma)
        
        if score > highest_score:
            highest_score = score
            associated_precision = precision_score(class_pred, three_sigma)
            associated_recall = recall_score(class_pred, three_sigma)
            associated_F1 = f1_score(class_pred, three_sigma)
            raise_by = j
        else:
            pass
    
    print("Best Training F1 Score:  ", associated_F1)
    print("Best Training Precision: ", associated_precision)
    print("Best Training Recall:    ", associated_recall)
    print('Stretchification Weight: ', raise_by)
    return raise_by
    

In [0]:
def run_model(cut, cut_2, reference, model=model, df=df):
    assert cut  > 0
    assert cut < 1
    assert cut_2 > 0
    assert cut_2 < 1
    
    
    X_train, X_test, y_train, y_test = split(cut)
    
    X_train_2, X_test_2, y_train_2, y_test_2 = second_split(cut_2,
                                                            X_train, 
                                                            X_test, 
                                                            y_train, 
                                                            y_test)
    
    print('################################################################')
    print('##################### TEST/TRAIN SPLITS ########################','\n')
    print('Shapes of Training Set:    ',X_train.shape, 
          X_test.shape, y_train.shape, y_test.shape)
    print('Shapes of Training Subset: ',X_train_2.shape, 
          X_test_2.shape, y_train_2.shape, y_test_2.shape,
          '\n\n')
    print('####################### SUBSET MODEL ###########################','\n')
    
    model.fit(X_train_2, y_train_2)
    
    y_pred = np.abs(model.predict(X_test_2))
    
    print("TrainingSubset R^2 Score:", r2_score(y_test_2, y_pred),'\n')
    
    weight = bootstrap_gradient(f1_score, X_train, X_test, y_train, y_test,
                                X_train_2, X_test_2, y_train_2, y_test_2,
                                y_pred)
    
    model.fit(X_train, y_train)
    
    final_prediction = model.predict(X_test)
    
    weighted_predict = (np.abs(final_prediction))**weight
    
    stdev = df['stdev'][int(len(df.index)*cut):len(df.index)].values
    class_pred_wei = (np.where(np.divide(weighted_predict, stdev) >= 3, 1, 0))
    three_sigma_val = df['3_sigma_event'][int(len(df.index)*cut):len(df.index)].values
    
    f1 = f1_score(three_sigma_val, class_pred_wei)
    prec = precision_score(three_sigma_val, class_pred_wei)
    rec = recall_score(three_sigma_val, class_pred_wei)
    
    if reference == 'final':
        print('\n\n##################### FINAL MODEL #############################','\n')
        print("Optimal F1:        " , f1)
        print("Optimal Precision: ", prec)
        print("Optimal Recall:    ", rec)
        return model

    else:
        return f1, prec, rec
    

In [0]:
def find_best():
    
    cut_range  = np.arange(start=.7, stop=.95, step=0.1)
    f1_scores  = []
    precision  = []
    recall     = []
    train_cut  = []
    subset_cut = []
    for i in cut_range:
        for j in cut_range: 
            with HiddenPrints():
                f1, prec, rec = run_model(i, j, reference='nope')
                
                f1_scores.append(f1)
                precision.append(prec)
                recall.append(rec)
                train_cut.append(i)
                subset_cut.append(j)
    
    results = pd.DataFrame([f1_scores, precision, recall,
                            train_cut, subset_cut])
    
    return results
                
                
                

In [0]:
results = find_best()

KeyError: ignored

In [0]:
cut = results.T.sort_values(0, ascending=False).iloc[0][3]
cut_2 = results.T.sort_values(0, ascending=False).iloc[0][4]

In [0]:
model = run_model(cut, cut_2, reference='final')

################################################################
##################### TEST/TRAIN SPLITS ######################## 

Shapes of Training Set:     (6385, 45) (710, 45) (6385,) (710,)
Shapes of Training Subset:  (5108, 45) (1277, 45) (5108,) (1277,) 


####################### SUBSET MODEL ########################### 

TrainingSubset R^2 Score: 0.648141317865416 

Best Training F1 Score:   0.7272727272727273
Best Training Precision:  0.7272727272727273
Best Training Recall:     0.7272727272727273
Stretchification Weight:  1.2349999999999741


##################### FINAL MODEL ############################# 

Optimal F1:         0.631578947368421
Optimal Precision:  0.6666666666666666
Optimal Recall:     0.6
